In [1]:
from bs4 import BeautifulSoup
import pandas as pd

## Read table from web page

In [2]:
html_doc = "index.html"
soup = BeautifulSoup(open(html_doc, encoding = "UTF-8"))

In [3]:
# retreive first table
ul = soup.find_all(name='table')
# print(ul[0])

items = []
for th in ul[0].find_all(name='td'):
    item = th.string
    if item is None: 
        for a in th.find_all(name="a"):
            item = a.string
            
    if item.endswith("\n"):
        item = item[:-1]
        
    items.append(item)
# print(items)

Postcode = [items[i] for i in range(0,len(items),3)]
Borough = [items[i] for i in range(1,len(items),3)]
Neighbourhood = [items[i] for i in range(2,len(items),3)]

## Pre-processing

In [4]:
p = list(set(Postcode))
p.sort(key = Postcode.index)
data_new = {"PostalCode":[],"Borough":[],"Neighborhood":[]}
for j in p:
    index = [i for i,x in list(enumerate(Postcode)) if Postcode[i] == j]
    #print(index)
    b = Borough[index[0]]
    if b != 'Not assigned': 
        n = ", ".join([Neighbourhood[i] for i in index])
        data_new["PostalCode"].append(j)
        data_new["Borough"].append(b)
        data_new["Neighborhood"].append(n)
        
# print(data_new)

## Convert to dataframe

In [5]:
df = pd.DataFrame(data_new)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


## Retreive coordinates

In [6]:
pc = pd.read_csv("Geospatial_Coordinates.csv")
# print(pc)

df["Latitude"] = ""
df["Longitude"] = ""

length = len(df)

for i in range(length):
    p = df["PostalCode"][i]
    lat = pc[pc["Postal Code"] == p]["Latitude"]
    lon = pc[pc["Postal Code"] == p]["Longitude"]
    df["Latitude"][i] = float(lat)
    df["Longitude"][i] = float(lon)

df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Downtown Toronto,Queen's Park,43.6623,-79.3895


In [7]:
df.shape

(103, 5)